In [1]:
import os
import requests
from dotenv import load_dotenv
import json
import re

In [2]:
# Charger les variables d'environnement
load_dotenv()
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

In [3]:
# Définir l'URL de l'API et les en-têtes
API_URL = "https://api-inference.huggingface.co/models/HuggingFaceH4/zephyr-7b-beta"
headers = {
    "Authorization": f"Bearer {HF_TOKEN}",
    "Content-Type": "application/json"
}

In [4]:


def get_definition_and_examples(word):
    # prompt 
    prompt = f"""
        You are an English language assistant. 
        Provide a JSON with a definition and two examples for the word "{word}".
        Return ONLY a valid JSON in this format:
        {{
            "definition": "brief definition here",
            "examples": ["example sentence 1", "example sentence 2"]
        }}
    """
    
    # Préparer la charge utile
    payload = {
        "inputs": prompt,
        "parameters": {
            "temperature": 0.1,  # Réduit pour des réponses plus cohérentes
            "max_new_tokens": 200,
            "return_full_text": False  # Ne retourne que le nouveau texte généré
        }
    }
    
    # Envoyer la requête POST
    response = requests.post(API_URL, headers=headers, json=payload)
    
    # Vérifier le statut de la réponse
    if response.status_code == 200:
        response_data = response.json()
        generated_text = response_data[0]["generated_text"]
        
        # Déboggage
        # print("Réponse complète:", generated_text)
        
        # Trouver tous les blocs qui ressemblent à du JSON
        all_json_candidates = re.findall(r'\{[^{]*"definition"[^{]*"examples"[^{]*\}', generated_text, re.DOTALL)
        
        # Si des candidats sont trouvés, essayer de les analyser un par un
        for json_candidate in all_json_candidates:
            try:
                # Nettoyer le candidat
                clean_json = json_candidate.strip()
                # Vérifier s'il y a du texte après la dernière accolade fermante
                if not clean_json.endswith('}'):
                    clean_json = clean_json[:clean_json.rfind('}')+1]
                
                # Essayer de parser le JSON
                result = json.loads(clean_json)
                
                # Vérifier que le résultat a la structure attendue
                if "definition" in result and "examples" in result and isinstance(result["examples"], list):
                    return result
            except json.JSONDecodeError:
                continue
        
        # Si aucun JSON valide n'a été trouvé, essayons d'extraire les informations manuellement
        definition_match = re.search(r'"definition"\s*:\s*"([^"]+)"', generated_text)
        examples_match = re.search(r'"examples"\s*:\s*\[\s*"([^"]+)"\s*,\s*"([^"]+)"\s*\]', generated_text)
        
        if definition_match and examples_match:
            return {
                "definition": definition_match.group(1),
                "examples": [examples_match.group(1), examples_match.group(2)]
            }
        
        print("Aucun JSON valide trouvé dans la réponse")
        return None
    else:
        print(f"Erreur API Hugging Face: {response.status_code} -> {response.text}")
        return None

In [5]:
# Exemple d'utilisation
word = "human being"
result = get_definition_and_examples(word)
if result:
    print("\nRésultat extrait avec succès:")
    print("Définition:", result["definition"])
    print("Exemples:", result["examples"])
else:
    print("Impossible d'obtenir la définition et les exemples.")


Résultat extrait avec succès:
Définition: A living organism belonging to the genus Homo and characterized by highly developed intelligence, articulate speech, and a contrarily frail body.
Exemples: ['She is a human being, just like you and me.', 'The human being has the ability to create and destroy.']


In [6]:


def get_definition_and_examples(word):
    # Structurer le prompt 
    prompt = f"""
        You are an English language assistant. 
        Provide a definition, French translation, and two example sentences in English for the English word "{word}". 
        Format your response as JSON with fields: word, definition, translation, example1, example2.
    """
    
    # Préparer la charge utile
    payload = {
        "inputs": prompt,
        "parameters": {
            "temperature": 0.1,  # Réduit pour des réponses plus cohérentes
            "max_new_tokens": 200,
            "return_full_text": False  # Ne retourne que le nouveau texte généré
        }
    }
    
    # Envoyer la requête POST
    response = requests.post(API_URL, headers=headers, json=payload)
    
    # Vérifier le statut de la réponse
    if response.status_code == 200:
        response_data = response.json()
        generated_text = response_data[0]["generated_text"]
        
        if generated_text : 
            return  json.loads(generated_text) 
        # if definition_match and examples_match:
        #     return {
        #         "definition": definition_match.group(1),
        #         "examples": [examples_match.group(1), examples_match.group(2)]
        #     }
        
        print("Aucun JSON valide trouvé dans la réponse")
        return None
    else:
        print(f"Erreur API Hugging Face: {response.status_code} -> {response.text}")
        return None

In [7]:
# Exemple d'utilisation
word = "human being"
result = get_definition_and_examples(word)
result

{'word': 'human being',
 'definition': 'A living organism belonging to the species Homo sapiens, characterized by intelligence, consciousness, and the ability to reason and learn through experience.',
 'translation': 'être humain',
 'example1': 'All human beings have the right to be treated with dignity and respect.',
 'example2': 'As human beings, we have a responsibility to protect the environment for future generations.'}

In [8]:
type(result)

dict

In [9]:
# 
word = "human being"
result = get_definition_and_examples(word)
if result:
    print("Définition:", result["definition"])
    print("Traduction:", result["translation"])
    print("Exemple 1:", result["example1"])
    print("Exemple 2:", result["example2"]) 
else:
    print("Impossible d'obtenir la définition et les exemples.")

Définition: A person, as opposed to an animal, plant, or other living thing.
Traduction: être humain
Exemple 1: As a human being, I have the right to be treated with dignity and respect.
Exemple 2: The human being's relationship with nature is complex and multifaceted.


In [6]:
import os
import requests
import json
import streamlit as st
from huggingface_hub import InferenceClient

# Récupère le token Hugging Face depuis les variables d'environnement
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

# Initialise le client avec un modèle warm
client = InferenceClient(
    model="mistralai/Mistral-Nemo-Instruct-2407",
    token=HF_TOKEN  # ou api_key=HF_TOKEN si version plus récente de huggingface_hub
)

def create_fallback_response(word):
    return {
        "word": word,
        "definition": "No definition available.",
        "translation": "Aucune traduction disponible.",
        "example1": "No example available.",
        "example2": "No example available."
    }

def extract_info_manually(text, word):
    # Extraction simple si le JSON est mal formé
    lines = text.split("\n")
    result = {"word": word}
    for line in lines:
        if ":" in line:
            key, val = line.split(":", 1)
            result[key.strip().lower()] = val.strip()
    return result

def get_definition_and_examples(word):
    if not HF_TOKEN:
        print("Token Hugging Face manquant. Veuillez configurer votre clé API.")
        return create_fallback_response(word)
    
    messages = [
        {"role": "system", "content": "You are an English assistant."},
        {"role": "user", "content": (
            f'Provide a definition, French translation, and two English example sentences for the word '
            f'"{word}". Format as JSON with keys: word, definition, translation, example1, example2.'
        )}
    ]

    try:
        response = client.chat.completions.create(
            model="mistralai/Mistral-Nemo-Instruct-2407",
            messages=messages,
            max_tokens=300,
            temperature=0.1,
        )
        text = response.choices[0].message.content

        try:
            return json.loads(text)
        except json.JSONDecodeError:
            return extract_info_manually(text, word)

    except Exception as e:
        print(f"Erreur lors de l'appel au modèle : {str(e)}")
        return create_fallback_response(word)


In [7]:
result = get_definition_and_examples("huge")
print(result)

{'word': 'huge', 'definition': 'extremely large in size or amount', 'translation': 'énorme', 'example1': 'The huge crowd filled the stadium to capacity.', 'example2': 'She has a huge collection of vintage records.'}
